In [19]:
#!pip install rdflib ipywidgets

In [21]:
import tkinter as tk
from tkinter import messagebox
import rdflib

In [23]:
# Load the ontology using RDFlib
def load_ontology():
    # Load your ontology (ensure to give the correct path)
    g = rdflib.Graph()
    g.parse("AreaCalculationOntology.owl", format="xml")
    return g

In [25]:

# Function to calculate area based on user input
def calculate_area(shape, base, height, side):
    try:
        # Check if the base, height, and side are not empty and convert to float
        if base:
            base = float(base)
        else:
            raise ValueError("Base cannot be empty")
        
        if height:
            height = float(height)
        else:
            height = 0  # Set height to 0 if not applicable (Square doesn't need height)
        
        if side:
            side = float(side)
        else:
            side = 0  # Set side to 0 if not applicable for other shapes

        # Area calculation logic based on shape
        if shape == "Rectangle":
            area = base * height
        elif shape == "Square":
            area = side ** 2
        elif shape == "Triangle":
            area = 0.5 * base * height
        else:
            raise ValueError("Unknown shape")
        return area

    except ValueError as e:
        messagebox.showerror("Input Error", f"Invalid input: {e}")
        return None

# Query the ontology for a property (e.g., sideLength or baseLength)
def query_ontology(g, property_name, class_name):
    query = f"""
    SELECT ?value WHERE {{
        ?shape rdf:type <{class_name}> .
        ?shape <{property_name}> ?value .
    }}
    """
    result = g.query(query)
    values = [str(row[0]) for row in result]
    return values

# GUI Setup
def setup_gui():
    # Create the main window
    window = tk.Tk()
    window.title("Area Calculator")

    # Load the ontology graph
    g = load_ontology()

    # Labels and Entry widgets for inputs
    shape_label = tk.Label(window, text="Select Shape:")
    shape_label.grid(row=0, column=0)
    
    shape_options = ["Rectangle", "Square", "Triangle"]
    shape_var = tk.StringVar(window)
    shape_var.set(shape_options[0])
    
    shape_menu = tk.OptionMenu(window, shape_var, *shape_options)
    shape_menu.grid(row=0, column=1)

    base_label = tk.Label(window, text="Base Length:")
    base_label.grid(row=1, column=0)
    base_entry = tk.Entry(window)
    base_entry.grid(row=1, column=1)

    height_label = tk.Label(window, text="Height Length:")
    height_label.grid(row=2, column=0)
    height_entry = tk.Entry(window)
    height_entry.grid(row=2, column=1)

    side_label = tk.Label(window, text="Side Length (if applicable):")
    side_label.grid(row=3, column=0)
    side_entry = tk.Entry(window)
    side_entry.grid(row=3, column=1)

    # Calculate button
    def on_calculate():
        shape = shape_var.get()
        base = base_entry.get()
        height = height_entry.get()
        side = side_entry.get()

        # Fetch relevant properties from ontology if applicable
        if shape == "Triangle":
            base_values = query_ontology(g, "#baseLength", "#Triangle")
            height_values = query_ontology(g, "#heightLength", "#Triangle")
        elif shape == "Rectangle":
            side_values = query_ontology(g, "#sideLength", "#Rectangle")
        elif shape == "Square":
            side_values = query_ontology(g, "#sideLength", "#Square")
        
        # Calculate and show area
        area = calculate_area(shape, base, height, side)
        if area is not None:
            result_label.config(text=f"Area: {area:.2f}")

    calculate_button = tk.Button(window, text="Calculate Area", command=on_calculate)
    calculate_button.grid(row=4, column=0, columnspan=2)

    # Label to display result
    result_label = tk.Label(window, text="Area: ")
    result_label.grid(row=5, column=0, columnspan=2)

    # Run the GUI loop
    window.mainloop()

# Run the application
if __name__ == "__main__":
    setup_gui()
